# 세션 4 – SLM vs LLM 비교

Foundry Local을 통해 실행되는 소형 언어 모델(SLM)과 대형 언어 모델(LLM) 간의 지연 시간 및 샘플 응답 품질을 비교합니다.


## ⚡ 빠른 시작

**메모리 최적화 설정 (업데이트됨):**
1. 모델이 CPU 변형을 자동으로 선택 (모든 하드웨어에서 작동)
2. `qwen2.5-3b`를 사용하여 7B 대신 사용 (~4GB RAM 절약)
3. 포트 자동 감지 (수동 설정 불필요)
4. 필요한 총 RAM: ~8GB 권장 (모델 + 운영체제)

**터미널 설정 (30초):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

그런 다음 이 노트북을 실행하세요! 🚀


### 설명: 의존성 설치
시간 측정 및 채팅 요청에 필요한 최소 패키지(`foundry-local-sdk`, `openai`, `numpy`)를 설치합니다. 안전하게 재실행할 수 있습니다.


# 시나리오
단일 프롬프트를 사용하여 대표적인 소형 언어 모델(SLM)과 대형 모델을 비교하여 다음과 같은 트레이드오프를 설명합니다:
- **지연 시간 차이** (실제 시간, 초 단위)
- **토큰 사용량** (사용 가능한 경우) - 처리량의 대리 지표로 사용
- **샘플 출력 품질** - 빠른 시각적 검토를 위해
- **속도 향상 계산** - 성능 향상을 정량화

**환경 변수:**
- `SLM_ALIAS` - 소형 언어 모델 (기본값: phi-4-mini, 약 4GB RAM)
- `LLM_ALIAS` - 대형 언어 모델 (기본값: qwen2.5-7b, 약 7GB RAM)
- `COMPARE_PROMPT` - 비교를 위한 테스트 프롬프트
- `COMPARE_RETRIES` - 복원력을 위한 재시도 횟수 (기본값: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - 서비스 엔드포인트 재정의 (설정되지 않은 경우 자동 감지)

**작동 방식 (공식 SDK 패턴):**
1. **FoundryLocalManager**가 Foundry Local 서비스를 초기화하고 관리합니다.
2. 서비스가 실행 중이 아니면 자동으로 시작됩니다 (수동 설정 불필요).
3. 모델은 별칭에서 구체적인 ID로 자동으로 해석됩니다.
4. 하드웨어 최적화된 변형이 선택됩니다 (CUDA, NPU 또는 CPU).
5. OpenAI 호환 클라이언트가 채팅 완료를 수행합니다.
6. 메트릭이 캡처됩니다: 지연 시간, 토큰, 출력 품질.
7. 결과를 비교하여 속도 향상 비율을 계산합니다.

이 미니 비교는 특정 사용 사례에서 더 큰 모델로 라우팅할 시점을 결정하는 데 도움을 줍니다.

**SDK 참조:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- 워크숍 유틸리티: ../samples/workshop_utils.py의 공식 패턴 사용

**주요 혜택:**
- ✅ 자동 서비스 검색 및 초기화
- ✅ 실행 중이 아니면 서비스 자동 시작
- ✅ 내장된 모델 해석 및 캐싱
- ✅ 하드웨어 최적화 (CUDA/NPU/CPU)
- ✅ OpenAI SDK 호환성
- ✅ 재시도를 통한 강력한 오류 처리
- ✅ 로컬 추론 (클라우드 API 불필요)


## 🚨 사전 준비 사항: Foundry Local이 실행 중이어야 합니다!

**이 노트북을 실행하기 전에**, Foundry Local 서비스가 설정되어 있는지 확인하세요:

### 빠른 시작 명령어 (터미널에서 실행):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### 대체 모델 (기본값이 사용 불가능한 경우):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **이 단계를 건너뛰면**, 아래 노트북 셀을 실행할 때 `APIConnectionError`가 발생할 수 있습니다.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### 설명: 핵심 가져오기
모델 정보를 가져오고 채팅 완료를 수행하는 데 사용되는 타이밍 유틸리티와 Foundry Local / OpenAI 클라이언트를 포함합니다.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### 설명: 별칭 및 프롬프트 설정
작은 모델과 더 큰 모델을 위한 환경 구성 가능한 별칭과 비교 프롬프트를 정의합니다. 환경 변수를 조정하여 다양한 모델 계열이나 작업을 실험할 수 있습니다.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 메모리 최적화 구성

**이 노트북은 기본적으로 메모리 효율적인 모델을 사용합니다:**
- `phi-4-mini` → 약 4GB RAM (Foundry Local은 CPU 변형을 자동 선택)
- `qwen2.5-3b` → 약 3GB RAM (7B는 ~7GB+가 필요)

**포트 자동 감지:**
- Foundry Local은 다른 포트를 사용할 수 있습니다 (주로 55769 또는 59959)
- 아래 진단 셀은 올바른 포트를 자동으로 감지합니다
- 수동 설정이 필요하지 않습니다!

**RAM이 제한적일 경우 (<8GB), 더 작은 모델을 사용하세요:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### 설명: 실행 도우미 (Foundry SDK 패턴)
워크숍 샘플에 문서화된 공식 Foundry Local SDK 패턴을 사용합니다:

**접근 방식:**
- **FoundryLocalManager** - Foundry Local 서비스를 초기화하고 관리
- **자동 감지** - 엔드포인트를 자동으로 탐지하고 서비스 라이프사이클을 처리
- **모델 해석** - 별칭을 전체 모델 ID로 해석 (예: phi-4-mini → phi-4-mini-instruct-cpu)
- **하드웨어 최적화** - 사용 가능한 하드웨어에 가장 적합한 변형 선택 (CUDA, NPU 또는 CPU)
- **OpenAI 클라이언트** - OpenAI 호환 API 액세스를 위해 매니저의 엔드포인트로 구성

**복원력 기능:**
- 지수적 백오프 재시도 로직 (환경 설정을 통해 구성 가능)
- 서비스가 실행 중이 아니면 자동으로 시작
- 초기화 후 연결 확인
- 상세한 오류 보고와 함께 우아한 오류 처리
- 반복 초기화를 방지하기 위한 모델 캐싱

**결과 구조:**
- 지연 시간 측정 (벽시계 시간 기준)
- 토큰 사용 추적 (가능한 경우)
- 샘플 출력 (가독성을 위해 축약됨)
- 실패한 요청에 대한 오류 세부 정보

이 패턴은 공식 SDK 패턴을 따르는 workshop_utils 모듈을 활용합니다.

**SDK 참조:**
- 메인 저장소: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- 워크숍 유틸리티: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### 설명: 사전 비행 자가 테스트
FoundryLocalManager를 사용하여 두 모델에 대해 간단한 연결 상태를 점검합니다. 이를 통해 다음을 확인합니다:
- 서비스에 접근 가능 여부
- 모델 초기화 가능 여부
- 별칭이 실제 모델 ID로 해석되는지 여부
- 비교 실행 전에 연결이 안정적인지 여부

setup() 함수는 workshop_utils의 공식 SDK 패턴을 사용합니다.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ 빠른 해결: 노트북에서 Foundry Local 시작하기 (선택 사항)

위 진단 결과 서비스가 실행 중이 아니라고 표시되면, 여기서 시작을 시도할 수 있습니다:

**참고:** 이 방법은 Windows에서 가장 잘 작동합니다. 다른 플랫폼에서는 터미널 명령을 사용하세요.


### ⚠️ 연결 오류 문제 해결

`APIConnectionError`가 표시된다면 Foundry Local 서비스가 실행 중이 아니거나 모델이 로드되지 않았을 수 있습니다. 다음 단계를 시도해 보세요:

**1. 서비스 상태 확인:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. 서비스 시작 (실행 중이 아닐 경우):**
```bash
foundry service start
```

**3. 필요한 모델 로드:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. 모델이 사용 가능한지 확인:**
```bash
foundry model ls
```

**일반적인 문제:**
- ❌ 서비스가 실행 중이 아님 → `foundry service start` 실행
- ❌ 모델이 로드되지 않음 → `foundry model run <model-name>` 실행
- ❌ 포트 충돌 → 다른 서비스가 포트를 사용 중인지 확인
- ❌ 방화벽 차단 → 로컬 연결이 허용되는지 확인

**빠른 해결 방법:** 사전 점검 전에 아래 진단 셀을 실행하세요.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ 사전 점검: 모델 가용성

이 셀은 비교를 실행하기 전에 두 모델이 설정된 엔드포인트에서 접근 가능한지 확인합니다.


### 설명: 실행 비교 및 결과 수집
공식 Foundry SDK 패턴을 사용하여 두 개의 별칭을 반복 처리합니다:
1. FoundryLocalManager를 사용하여 각 모델을 setup()으로 초기화합니다.
2. OpenAI 호환 API를 사용하여 추론을 실행합니다.
3. 지연 시간, 토큰, 샘플 출력 데이터를 캡처합니다.
4. 비교 분석이 포함된 JSON 요약을 생성합니다.

이는 session04/model_compare.py에 있는 워크숍 샘플과 동일한 패턴을 따릅니다.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### 결과 해석

**핵심 지표:**
- **지연 시간**: 낮을수록 좋음 - 더 빠른 응답 시간을 나타냄
- **토큰**: 처리량이 높을수록 더 많은 토큰이 처리됨
- **경로**: 사용된 API 엔드포인트를 확인

**SLM과 LLM을 언제 사용할지:**
- **SLM (소형 언어 모델)**: 빠른 응답, 낮은 자원 사용, 간단한 작업에 적합
- **LLM (대형 언어 모델)**: 높은 품질, 더 나은 추론, 품질이 가장 중요한 경우 사용

**다음 단계:**
1. 다양한 프롬프트를 시도하여 복잡성이 비교에 미치는 영향을 확인
2. 다른 모델 쌍을 실험
3. 작업 복잡성에 따라 지능적으로 라우팅하기 위해 워크숍 라우터 샘플(Session 06)을 사용


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 노력하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서를 해당 언어로 작성된 상태에서 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
